# Create Data Tables For ONT Data QC<a class="tocSkip">

**This notebook automatically reads in data stored in the AnVIL_HPRC workspace's bucket and generates data table so the data can be run through QC (NTSM and ReadStat's WDLs)**

**Ivo changed this in Sep 2023 to output per flowcell ONT Seq Summary**

**Jeltje changed this in Feb 2023 to output per-file tables instead of per-sample**

**Below are the steps taken in this notebook:**
1. Import Statements & Global Variable Definitions
2. Define Functions
3. Read In Sample Names
4. Create Dataframe Of Files
5. Write data frame to data tables

# Import Statements & Global Variable Definitions

## Installs

In [ ]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [ ]:
%%capture
%pip install terra-pandas
#%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [ ]:
from firecloud import fiss
import pandas as pd         
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations
Update this with project specific info

In [ ]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

sample_info_fn = 'UCSC_HPRC_ONT_Y3_GUPPY6_samples.tsv' # a list of IDs,e.g. HG01243
subm_key       = '79275EDA-C282-424A-9D5B-A8E876592893--UCSC_HPRC_ONT_Y3_GUPPY6'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

# Function Definitions

In [ ]:
def rtn_datatype_ls_for_subm(bucket_url, submission_key, data_type_subdir, file_type_ls):
    '''Takes in:
            * bucket_url (string): url of bucket to search (should be the AnVIL_HPRC bucket)
                ex: "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"
            * submission_key (string): the UUID plus the submission name to search
                ex: "5c68b972-8534-402f-9861-11c93558765f--UW_HPRC_HiFi_Y3"
            * data_type_subdir (string): name of the subfolder to search 
              (used when a submission has more than one data type.)
                ex: "PacBio_HiFi" if the data is not in subfolders, pass "."
            * file_type_ls (list of strings): file extensions to search for. Often a submission will 
              have more than one type of file that represents the same dataset. Be careful to not 
              include replicate data, however.
                ex: ".hifi_reads.bam"
     then performs a list of the bucket, then returns a filtered list files.'''
    
    rtn_file_ls = []
    
    submission_path = str(bucket_url + "submissions/" + submission_key)
    file_list_byt   = subprocess.run(['gsutil', '-u', 'human-pangenome-ucsc', 'ls', '-r', submission_path], 
                                     stdout=subprocess.PIPE)

    file_list_str   = file_list_byt.stdout.decode('utf-8')
    file_list       = file_list_str.split('\n')  ## Pull out "\n"
   
    ## filter out empty strings
    file_list = [ elem for elem in file_list if elem != '']
    
    ## Pull from subdir type we are targeting
    file_list = list(filter(lambda x:re.search(rf"{data_type_subdir}", x), file_list))
    
    for file_type in file_type_ls:
    
        ## Pull files of correct type (ex: ccs.bam)
        file_list_by_type = list(filter(lambda x:re.search(rf"{file_type}$", x), file_list))

        ## Add to list of files to return
        rtn_file_ls += file_list_by_type

    return rtn_file_ls    

# Read In Sample Names

In [ ]:
## This file (a list of sample IDs) should be stored in the Terra workspace (in the bucket) that is being used for 
## the submission you  are wrangling...
sample_info_fp = os.path.join(bucket + sample_info_fn) 

sample_df = pd.read_csv(sample_info_fp, header=None)

sample_df.rename(columns = {0:'sample_id'}, inplace = True)
sample_df.head()

# Create Dataframe Of Files

## Find the data

In [ ]:
data_type_subdir = "."
file_type_ls     = ["sequencing_summary.txt.gz"]

## get a list of the files in submission (subm_key) that match the 
## data_type_subdir and file_type_ls
file_ls  = rtn_datatype_ls_for_subm(anvil_hprc_bucket, subm_key, 
                                      data_type_subdir, file_type_ls)

## Check that the number of files matches what we expect
len(file_ls)


## Add Sample Data To File Data Frame

In [ ]:
filedict = dict()

for f in file_ls:
    for s in sample_df.sample_id:
        if re.search(s,f):
            filedict[f] = s
            break            
print(len(filedict))
file_df = pd.DataFrame.from_dict(filedict.items())
file_df.columns = ['ONT_seq_summ', 'sample']
file_df.head()


In [ ]:
print(file_df.shape)

# Upload To Tables

In [ ]:
## Create tables for running NTSM and ReadStats
dataframe_to_table("covstats", file_df, WORKSPACE, PROJECT)